In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [ ]:
import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=5100)
import pickle
datafile= open('/Users/barrybrent/run10may23.txt','wb') 
data=[]
start=time.time()/60
for a in [1..5000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()
#-----------------------------------------------------

import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run10may23no2.txt','wb') 
data=[]
start=time.time()/60
for a in [5001..7000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [3]:
a=[[1],[2]];b=[[3],[4]];c=a+b;print(c);
data=[]
for tpl in c:data=data+tpl
print(data)

[[1], [2], [3], [4]]
[1, 2, 3, 4]


In [6]:
import pickle
rfile1=open('/Users/barrybrent/run10may23.txt','rb') 
rs1=pickle.load(rfile1)
rfile1.close
s1=stripQuotationMarks(rs1)

rfile2=open('/Users/barrybrent/run10may23no2.txt','rb') 
rs2=pickle.load(rfile2)
rfile2.close
s2=stripQuotationMarks(rs2)

datafile= open('/Users/barrybrent/run12may23.txt','wb') 
data=[]
s=s1+s2
for tpl in s:
    data=data+[tpl] # contrary to the cell above, Sage insisted on the brackets in this one.
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [7]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
print(s[0]);print(s[1]);
print(len(s))


(1, -3)
(2, 19)
7000


In [12]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)


for k in [0..10]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    ds=digitsum(2,a)
    print((a,ds,o2))
    

(1, 1, 0)
(2, 1, 0)
(3, 2, 0)
(4, 1, 0)
(5, 2, 0)
(6, 2, 0)
(7, 3, 0)
(8, 1, 0)
(9, 2, 0)
(10, 2, 0)
(11, 3, 0)


In [11]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    data=data+[o2]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)


for k in [0..10]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    ds3=digitsum(3,a)
    print((a,ds3,o3))
    

(1, 1, 1)
(2, 2, 0)
(3, 1, 1)
(4, 2, 0)
(5, 3, 1)
(6, 2, 0)
(7, 3, 1)
(8, 4, 0)
(9, 1, 1)
(10, 2, 0)
(11, 3, 3)


In [15]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..60]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if o3==0:data=data+[a]
print(data)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60]


In [17]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if o3==0:
        if mod(a,2)==0:
            yes=yes+[a]
        if mod(a,2)!=0:
            no=no+[a]
print((len(yes),len(no)))

(3500, 0)


In [19]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==0:
        c=pair[1]
        o3=order(3,c)
        if o3==0:yes=yes+[a]
        if o3!=0:no=no+[a]
print((len(yes),len(no)))

(3500, 0)


In [41]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==0:
        c=pair[1]
        o3=order(3,c)
        sk=c/3^o3
        m3=mod(sk,3)
        if m3==1:yes=yes+[a]
        if m3!=1:no=no+[a]
print((len(yes),len(no)))

(3500, 0)


In [20]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

rd3=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    rd3=rd3+[o3]
ssr=sortAndSmash(rd3)
print(ssr)
    
    
    

[0, 1, 2, 3, 4, 5, 6, 8]


In [22]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if o3==1:data=data+[a]
print(data) #no OEIS listing

[1, 3, 5, 7, 9, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 37, 39, 43, 45, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 109, 111, 115, 117, 121, 123, 127, 129, 133, 135, 139, 141, 145, 147, 151, 153, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 325, 327, 331, 333, 337, 339, 343, 345, 349, 351, 355, 357, 361, 363, 367, 369, 373, 375, 379, 381, 385, 387, 391, 393, 397, 399, 403, 405, 409, 411, 415, 417, 421, 423, 427, 429, 433, 435, 439, 441, 445, 447, 451, 453, 457, 459, 463, 465, 469, 471, 475, 477, 481, 483, 485, 487, 489, 491, 493, 495, 497, 499, 

In [23]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if not(is_prime(a)):
        c=pair[1]
        o3=order(3,c)
        if o3==1:data=data+[a]
print(data) # no OEIS entry

[1, 9, 15, 21, 25, 27, 33, 39, 45, 49, 51, 55, 57, 63, 65, 69, 75, 77, 81, 85, 87, 91, 93, 95, 99, 105, 111, 115, 117, 121, 123, 129, 133, 135, 141, 145, 147, 153, 159, 161, 165, 169, 171, 175, 177, 183, 185, 187, 189, 195, 201, 203, 205, 207, 209, 213, 215, 217, 219, 221, 225, 231, 235, 237, 243, 245, 247, 249, 253, 255, 259, 261, 265, 267, 273, 275, 279, 285, 287, 289, 291, 295, 297, 299, 301, 303, 305, 309, 315, 319, 321, 325, 327, 333, 339, 343, 345, 351, 355, 357, 361, 363, 369, 375, 381, 385, 387, 391, 393, 399, 403, 405, 411, 415, 417, 423, 427, 429, 435, 441, 445, 447, 451, 453, 459, 465, 469, 471, 475, 477, 481, 483, 485, 489, 493, 495, 497, 501, 505, 507, 511, 513, 515, 517, 519, 525, 527, 529, 531, 533, 535, 537, 539, 543, 545, 549, 551, 553, 555, 559, 561, 565, 567, 573, 575, 579, 581, 583, 585, 589, 591, 595, 597, 603, 605, 609, 611, 615, 621, 623, 625, 627, 629, 633, 635, 637, 639, 645, 649, 651, 655, 657, 663, 665, 667, 669, 671, 675, 679, 681, 685, 687, 689, 693, 695, 6

In [42]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if is_prime(a):
        c=pair[1]
        o3=order(3,c)
        data=data+[o3]
print(data) 

[0, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 4, 2, 1, 3, 1, 2, 1, 2, 1, 1, 3, 2, 1, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 3, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 4, 1, 1, 2, 3, 2, 1, 1, 1, 4, 3, 1, 2, 2, 1, 1, 2, 1, 3, 2, 1, 1, 5, 2, 2, 1, 1, 2, 1, 2, 1, 3, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [44]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if is_prime(a):
        c=pair[1]
        o3=order(3,c)
        if o3>1:data=data+[a]
print(data) 

[11, 41, 47, 107, 113, 131, 137, 149, 347, 353, 359, 383, 389, 401, 419, 431, 443, 449, 461, 467, 479, 971, 977, 983, 1013, 1019, 1031, 1049, 1061, 1091, 1097, 1103, 1109, 1151, 1163, 1181, 1187, 1193, 1217, 1223, 1229, 1259, 1277, 1283, 1289, 1301, 1307, 1319, 1361, 1367, 1373, 1409, 1427, 1433, 1439, 1451, 2927, 2939, 2957, 2963, 2969, 2999, 3011, 3023, 3041, 3083, 3089, 3119, 3137, 3167, 3191, 3203, 3209, 3221, 3251, 3257, 3299, 3323, 3329, 3347, 3359, 3371, 3389, 3407, 3413, 3449, 3461, 3467, 3491, 3527, 3533, 3539, 3557, 3581, 3593, 3617, 3623, 3659, 3671, 3677, 3701, 3719, 3761, 3767, 3779, 3797, 3803, 3821, 3833, 3851, 3863, 3881, 3911, 3917, 3923, 3929, 3947, 3989, 4001, 4007, 4013, 4019, 4049, 4073, 4079, 4091, 4127, 4133, 4139, 4157, 4211, 4217, 4229, 4241, 4253, 4259, 4271, 4283, 4289, 4337, 4349]


In [45]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if is_prime(a):
        c=pair[1]
        o3=order(3,c)
        if o3==2:data=data+[a]
print(data) 

[107, 113, 137, 149, 347, 353, 389, 419, 431, 449, 467, 971, 977, 983, 1013, 1019, 1031, 1049, 1091, 1103, 1109, 1151, 1163, 1181, 1187, 1217, 1229, 1283, 1289, 1301, 1319, 1367, 1373, 1409, 1427, 1439, 2927, 2939, 2957, 2963, 2999, 3011, 3083, 3089, 3119, 3137, 3191, 3209, 3251, 3299, 3323, 3359, 3371, 3389, 3413, 3449, 3467, 3491, 3527, 3539, 3557, 3581, 3593, 3617, 3659, 3677, 3701, 3719, 3767, 3803, 3821, 3863, 3881, 3911, 3923, 3929, 3947, 4001, 4013, 4019, 4049, 4073, 4091, 4127, 4139, 4157, 4211, 4217, 4229, 4253, 4271, 4283, 4289, 4337]


In [27]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,3)==0:
        c=pair[1]
        o3=order(3,c)
        if o3==1:
            ad=a.digits(3)
            print((a,ad))
            data=data+[a]
print();print(data) # A016945		a(n) = 6*n+3.


(3, [0, 1])
(9, [0, 0, 1])
(15, [0, 2, 1])
(21, [0, 1, 2])
(27, [0, 0, 0, 1])
(33, [0, 2, 0, 1])
(39, [0, 1, 1, 1])
(45, [0, 0, 2, 1])
(51, [0, 2, 2, 1])
(57, [0, 1, 0, 2])
(63, [0, 0, 1, 2])
(69, [0, 2, 1, 2])
(75, [0, 1, 2, 2])
(81, [0, 0, 0, 0, 1])
(87, [0, 2, 0, 0, 1])
(93, [0, 1, 1, 0, 1])
(99, [0, 0, 2, 0, 1])
(105, [0, 2, 2, 0, 1])
(111, [0, 1, 0, 1, 1])
(117, [0, 0, 1, 1, 1])
(123, [0, 2, 1, 1, 1])
(129, [0, 1, 2, 1, 1])
(135, [0, 0, 0, 2, 1])
(141, [0, 2, 0, 2, 1])
(147, [0, 1, 1, 2, 1])
(153, [0, 0, 2, 2, 1])
(159, [0, 2, 2, 2, 1])
(165, [0, 1, 0, 0, 2])
(171, [0, 0, 1, 0, 2])
(177, [0, 2, 1, 0, 2])
(183, [0, 1, 2, 0, 2])
(189, [0, 0, 0, 1, 2])
(195, [0, 2, 0, 1, 2])
(201, [0, 1, 1, 1, 2])
(207, [0, 0, 2, 1, 2])
(213, [0, 2, 2, 1, 2])
(219, [0, 1, 0, 2, 2])
(225, [0, 0, 1, 2, 2])
(231, [0, 2, 1, 2, 2])
(237, [0, 1, 2, 2, 2])
(243, [0, 0, 0, 0, 0, 1])
(249, [0, 2, 0, 0, 0, 1])
(255, [0, 1, 1, 0, 0, 1])
(261, [0, 0, 2, 0, 0, 1])
(267, [0, 2, 2, 0, 0, 1])
(273, [0, 1, 0, 1, 0, 1

In [28]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==3:
        c=pair[1]
        o3=order(3,c)
        if o3==1:yes=yes+[a]
        if o3!=1:no=no+[a]
print((len(yes),len(no)))
            

(117, 0)


In [36]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if  o3==1:
        if ZZ(mod(a,6))==3:
            yes=yes+[a]
        if ZZ(mod(a,6))!=3:no=no+[a]
print((len(yes),len(no)))

(117, 193)


In [31]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==1:
        c=pair[1]
        o3=order(3,c)
        data=data+[o3]
print(data)
        

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [34]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==5:
        c=pair[1]
        o3=order(3,c)
        data=data+[o3]
print(data)

[1, 3, 1, 1, 1, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 4, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [35]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==5:
        c=pair[1]
        o3=order(3,c)
        if o3==1:
            data=data+[a]
print(data)

[5, 17, 23, 29, 53, 59, 65, 71, 77, 83, 89, 95, 101, 161, 167, 173, 179, 185, 191, 197, 203, 209, 215, 221, 227, 233, 239, 245, 251, 257, 263, 269, 275, 281, 287, 293, 299, 305, 311, 317, 485, 491, 497, 503, 509, 515, 521, 527, 533, 539, 545, 551, 557, 563, 569, 575, 581, 587, 593, 599, 605, 611, 617, 623, 629, 635, 641, 647, 653, 659, 665, 671, 677, 683, 689, 695]


In [38]:
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)!=0:
        if mod(a,6)==5:
            c=pair[1]
            o3=order(3,c)
            data=data+[o3]
print(data)

[1, 3, 1, 1, 1, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 4, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [46]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==1:
        c=pair[1]
        o3=order(3,c)
        sk=c/3^o3
        m3=mod(sk,3)
        data=data+[m3]
print(data)

[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [ ]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    
    c=pair[1]
    o3=order(3,c)
    sk=c/3^o3
    m3=mod(sk,3)
    if m3==1:data=data+[a]
print(data)

In [50]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    
    c=pair[1]
    o3=order(3,c)
    sk=c/3^o3
    m3=mod(sk,3)
    if m3==2:data=data+[a]
print(data)

[1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 39, 41, 45, 47, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 111, 117, 119, 123, 125, 129, 135, 137, 141, 147, 149, 153, 155, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 327, 333, 335, 339, 341, 345, 351, 353, 357, 359, 363, 369, 371, 375, 377, 381, 383, 387, 393, 395, 399, 405, 407, 411, 413, 417, 423, 425, 429, 435, 437, 441, 443, 447, 453, 455, 459, 461, 465, 471, 473, 477, 479, 483, 485, 487, 489, 491, 493, 495, 497, 499, 501, 503, 505, 507, 509, 511, 513, 515, 517, 519, 521, 523, 525, 527, 529,

In [9]:
data2=[]
data3=[]
for j in [0..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
    o2=order(2,rsp)
    data2=data2+[o2]
    o3=order(3,rsp)
    data3=data3+[o3]
print(data2);print();print(data3)



[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 4, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 5, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0]


In [15]:

data2=[]
data3=[]
for j in [0..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
   
    o3=order(3,rsp)
    if mod(j,2)==0:
        print((j,o3))
        


(0, 0)
(2, 0)
(4, 0)
(6, 0)
(8, 0)
(10, 0)
(12, 0)
(14, 0)
(16, 0)
(18, 0)
(20, 0)
(22, 0)
(24, 0)
(26, 0)
(28, 0)
(30, 0)
(32, 0)
(34, 0)
(36, 0)
(38, 0)
(40, 0)
(42, 0)
(44, 0)
(46, 0)
(48, 0)
(50, 0)
(52, 0)
(54, 0)
(56, 0)
(58, 0)
(60, 0)
(62, 0)
(64, 0)
(66, 0)
(68, 0)
(70, 0)
(72, 0)
(74, 0)
(76, 0)
(78, 0)
(80, 0)
(82, 0)
(84, 0)
(86, 0)
(88, 0)
(90, 0)
(92, 0)
(94, 0)
(96, 0)
(98, 0)
(100, 0)


In [21]:
data=[]
for j in [2..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
   
    o3=order(3,rsp)
    if mod(j,2)==1:
        data=data+[order(3,j)+1-o3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
data=[]
for j in [2..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(2)
   
    o3=order(3,rsp)
    if mod(j,2)==1:
        data=data+[o3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
for power in [1..10]:
    data=[]
    for j in [2..100]:
        def Sigma(n):
            ans=0
            for a in [1..n+1]:
                term=sigma(a,k=j) 
                ans=ans+term*x^a
            return D(ans)


        def recipSigma(n):
            g=L(1/Sigma(2*n))
            return g

        def recipSigmaPowerConstant(n):
            f=recipSigma(n)^n
            return f[0]
    
        rsp=recipSigmaPowerConstant(power)
   
        o3=order(3,rsp)
        if mod(j,2)==1:
            data=data+[o3]
    print("power",power);print(data)

power 1
[2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 2, 1, 1, 3]
power 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 3
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
power 4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 5
[1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1]
power 6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 7
[2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2